In [17]:
import pymongo
import os
import json
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
_URI = f"mongodb://{os.environ['MONGODB_USERNAME']}:{os.environ['MONGODB_PASSWORD']}@{os.environ['MONGODB_HOST']}:{os.environ['MONGODB_PORT']}/{os.environ['MONGODB_DATABASE']}"

In [19]:
client = pymongo.MongoClient(_URI)
client.list_database_names()

['admin', 'config', 'function_calling', 'local', 'mqtt_logs']

In [20]:
db = client["function_calling"]
collection = db["gorilla_openfunctions"]

In [21]:
def recursively_dump_to_json(input_json_str):
    # Parse the outer JSON string
    input_json_str = str(input_json_str)
    input_json_str = input_json_str.strip()
    try:
        outer_data = json.loads(input_json_str)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {
            "error": str(e),
            "data": input_json_str
        }

    # Function to recursively convert inner JSON strings
    def convert_inner_json(obj):
        if isinstance(obj, str):
            obj = obj.strip()
            if obj.startswith("{") and obj.endswith("}"):
                try:
                    return json.loads(obj)
                except json.JSONDecodeError as e:
                    print(f"Error parsing JSON: {e}")
                    raise e
            elif obj.startswith("[") and obj.endswith("]"):
                try:
                    return eval(obj)
                except Exception as e:
                    print(f"Error evaluating List: {e}")
                    raise e
            
            return obj
            
        elif isinstance(obj, list):
            return [convert_inner_json(item) for item in obj]
        
        elif isinstance(obj, dict):
            return {key: convert_inner_json(value) for key, value in obj.items()}

    # Recursively convert inner JSON strings
    converted_data = convert_inner_json(outer_data)
    
    return dict(converted_data)

In [26]:
with open(os.environ["TEST_FILE_PATH"]) as f:
    for num, data in enumerate(f.readlines()):
        
        if data.strip() == "":
            continue
        
        # if collection.count_documents({"split": "test", "line": num+1}) > 0:
        #     collection.delete_many({"split": "test", "line": num+1})

        data = recursively_dump_to_json(data)
        
        try:
            collection.insert_one(
                {
                    "split": "test",
                    "line": num+1,
                    "data": data
                }
            )
        except pymongo.errors.DuplicateKeyError:
            print(f"Duplicate key error at line {num+1}")
            # Get the duplcate element
            duplicate_element = collection.find_one({"split": "test", "line": num+1})
            if duplicate_element["data"] != data:
                print("Duplicate element is different")
                print(f"Old element: {duplicate_element}")
                print(f"New element: {data}")
                print("Deleting old element and inserting new element")
                collection.delete_one({"split": "test", "line": num+1})
                collection.insert_one(
                    {
                        "split": "test",
                        "line": num+1,
                        "data": data
                    }
                )
            else:
                print("Duplicate element is same, skipping")

Duplicate key error at line 1
Duplicate element is same, skipping
Duplicate key error at line 2
Duplicate element is same, skipping
Duplicate key error at line 3
Duplicate element is same, skipping
Duplicate key error at line 4
Duplicate element is same, skipping
Duplicate key error at line 5
Duplicate element is same, skipping
Duplicate key error at line 6
Duplicate element is same, skipping
Duplicate key error at line 7
Duplicate element is same, skipping
Duplicate key error at line 8
Duplicate element is same, skipping
Duplicate key error at line 9
Duplicate element is same, skipping
Duplicate key error at line 10
Duplicate element is same, skipping
Duplicate key error at line 11
Duplicate element is same, skipping
Duplicate key error at line 12
Duplicate element is same, skipping
Duplicate key error at line 13
Duplicate element is same, skipping
Duplicate key error at line 14
Duplicate element is same, skipping
Duplicate key error at line 15
Duplicate element is same, skipping
Dupl

In [25]:
with open(os.environ["TRAIN_FILE_PATH"]) as f:
    for num, data in enumerate(f.readlines()):
        
        if data.strip() == "":
            continue
        
        if collection.count_documents({"split": "train", "line": num+1}) > 0:
            collection.delete_many({"split": "train", "line": num+1})
            
        # collection.insert_one(
        #     {
        #         "split": "train",
        #         "line": num+1,
        #         "raw_data": recursively_dump_to_json(data)
        #     }
        # )
        print(num+1, data.strip())
        recursively_dump_to_json(data)

1 {"Instruction": "I want to create a one-dimensional tensor with evenly spaced values from 0 to 1 using the torch.linspace API.\n", "Functions": ["{'name': 'Torch', 'api_name': 'torch.linspace', 'description': 'Create a one-dimensional tensor with evenly spaced values', 'parameters': {'start': {'type': 'float', 'description': 'The starting value for the set of points'}, 'end': {'type': 'float', 'description': 'The ending value for the set of points'}, 'steps': {'type': 'int', 'description': 'The number of evenly spaced values to generate'}, 'out': {'type': 'Tensor', 'description': 'Optional output tensor'}, 'dtype': {'type': 'torch.dtype', 'description': 'Optional data type for the computation'}, 'layout': {'type': 'torch.layout', 'description': 'Optional layout of the returned tensor'}, 'device': {'type': 'torch.device', 'description': 'Optional device for the returned tensor'}, 'requires_grad': {'type': 'bool', 'description': 'Optional flag to enable gradient tracking'}}}\n", "{'nam

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)